In [29]:
import torch
import torchvision
import pandas as pd
import os
from tqdm import tqdm
import pydicom
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from utils import normalize_dicom

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torchvision.models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)  
model = model.to(device)

model.load_state_dict(torch.load('frame_selection.pth'))

for param in model.parameters():
    param.requires_grad = False

In [212]:
df1 = pd.read_csv("data/v2/imagerejectionfeedback.csv", sep=";")
df1 = df1.set_index("IMAGE_ID")
#df = df.set_index("IMAGE_ID")
df2 = pd.read_csv("data/v2/image.csv", sep=";")
df2 = df2.set_index("ID")

df = df1.join(df2, how="inner")

test_img_ids = pd.read_csv("testimgs.csv")
test_img_ids = set(test_img_ids.ID)

df = df[df.index.isin(test_img_ids)]
df

,ID,TECHNICALVESSELSNOTVISIBLE,TECHNICALVESSELSOVERLAP,TECHNICALNOCONTRAST,TECHNICALLOWQUALITY,PROCEDURALFEMORALACCESS,PROCEDURALELECTRODESVISIBLE,PROCEDURALCATHETERINAORTA,EXAMINATIONAORTOGRAPHY,EXAMINATIONVENTRICULOGRAPHY,...,POSITIONERSECONDARYANGLEFRAMEOFFSET,REPRESENTATIVEFRAME,SERIESDESCRIPTION,SERIESINSTANCEUID,SERIESNO,SEX,STUDYDESCRIPTION,STUDYINSTANCEUID,URL,STUDY_ID
IMAGE_ID,,,,,,,,,,,,,,,,,,,,,
12nc2fo321j31atm78c97qppv9sb,18645444-f508-4800-940d-d745767fb10e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],0,NaN,1.2.840.113619.2.55.3.2831188764.331.161531790...,1,M,TK KLATKA PIERSIOWA BE,1.3.6.1.4.1.22410.3166677045851423856869482016...,Rzeszow-MSW2/7/IM_1563,1.3.6.1.4.1.22410.3166677045851423856869482016...
12nc2fo321j31atm78c97qppv9se,7f105bd8-b80b-45c2-86f3-3afaaa9edd80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],0,NaN,1.2.840.113619.2.55.3.2831188764.331.161531790...,2,M,TK KLATKA PIERSIOWA BE,1.3.6.1.4.1.22410.3166677045851423856869482016...,Rzeszow-MSW2/7/IM_1454,1.3.6.1.4.1.22410.3166677045851423856869482016...
12nc2fo321j31atm78c97qppv9se3,ee1c8ee3-3ef1-47d8-b887-bae01cba7713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],0,Recon 2:,1.2.840.113619.2.55.3.2831188764.331.161531790...,3,M,TK KLATKA PIERSIOWA BE,1.3.6.1.4.1.22410.3166677045851423856869482016...,Rzeszow-MSW2/7/IM_1395,1.3.6.1.4.1.22410.3166677045851423856869482016...
12nc2fo3228g3fg5sijsqpyazd141,240b4f99-1f16-468a-bae9-ecefce585908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],0,Processed Images,1.2.840.113619.2.80.973218448.24040.1615711513...,200,M,TK KLATKA PIERSIOWA BE,1.3.6.1.4.1.22410.3166677045851423856869482016...,Rzeszow-MSW2/7/IM_1199,1.3.6.1.4.1.22410.3166677045851423856869482016...
12nc2fo3228g3fg5sijsqpyazow41,87cc59e4-abb3-4746-89c5-457dc5463e8d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],0,Processed Images,1.2.840.113619.2.80.973218448.24040.1615711524...,201,M,TK KLATKA PIERSIOWA BE,1.3.6.1.4.1.22410.3166677045851423856869482016...,Rzeszow-MSW2/7/IM_1154,1.3.6.1.4.1.22410.3166677045851423856869482016...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13c2ur549vohc0jat2ewj4eycc2,dcd19103-5106-4f34-96a7-4a573b39fb81,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],38,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,1,M,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,data/05/Coronary_Diagnostic_Coronary_Catheteri...,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...
13c2ur549vohc0jat2ewj4eyk51,f4673173-4331-4c25-9c5c-889ced6a24d0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],33,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,3,M,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,data/05/Coronary_Diagnostic_Coronary_Catheteri...,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...
13c2ur549vohc0jat2ewk4eyu41,da52624d-f7c2-4977-ac37-acc1c1b66577,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[],35,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112628.7416...,6,M,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,data/05/Coronary_Diagnostic_Coronary_Catheteri...,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...
